# TensorFlow 2 quickstart for beginners

* Go through the [TensorFlow 2.x quick start for beginners](https://www.tensorflow.org/tutorials/quickstart/beginner) tutorial (be sure to type out all of the code yourself, even if you don't understand it).

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

## Setup TensorFlow

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

2024-10-13 14:38:36.026813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 14:38:41.016714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-13 14:38:42.745073: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-13 14:38:43.222273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-13 14:38:46.837152: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.17.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [17]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.06276149, -0.15934546,  0.16248721, -0.9003595 ,  0.1523379 ,
        -0.4256401 , -0.31988165,  0.5192411 ,  0.9743914 , -0.04815859]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [18]:
tf.nn.softmax(predictions).numpy()

array([[0.09399676, 0.07527541, 0.103854  , 0.03587854, 0.10280529,
        0.05767702, 0.06411108, 0.14837457, 0.2338992 , 0.08412808]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [19]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [28]:
loss_fn(y_train[:1], predictions).numpy()

2.8528967

In [26]:
-tf.math.log(1/10)

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025851>

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [21]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [22]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8594 - loss: 0.4821
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9558 - loss: 0.1507
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9666 - loss: 0.1099
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9729 - loss: 0.0876
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9762 - loss: 0.0752


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [23]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - 5ms/step - accuracy: 0.9764 - loss: 0.0787


[0.07871416211128235, 0.9764000177383423]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [24]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.5689596e-08, 1.1480618e-08, 7.3399870e-06, 3.5270580e-04,
        2.1221884e-11, 1.5059203e-06, 6.1283617e-12, 9.9963629e-01,
        7.0791074e-08, 2.0116108e-06],
       [1.0839389e-07, 5.5351126e-04, 9.9935800e-01, 2.2748121e-05,
        4.1428398e-15, 6.1851075e-05, 1.3585615e-06, 1.4548162e-13,
        2.3340906e-06, 2.1568554e-10],
       [7.8169585e-08, 9.9907577e-01, 2.1902693e-04, 1.2123747e-05,
        5.8818468e-06, 2.6106884e-06, 7.0574117e-07, 4.0355625e-04,
        2.7835093e-04, 1.7736021e-06],
       [9.9991524e-01, 2.6289498e-11, 3.6501779e-06, 3.1994885e-09,
        1.6738521e-07, 2.0459440e-08, 1.5588261e-05, 7.1690397e-07,
        2.4335176e-09, 6.4608888e-05],
       [6.8242576e-07, 2.1926708e-08, 4.0282612e-06, 4.4085745e-08,
        9.9414128e-01, 2.1245226e-06, 1.4770305e-05, 1.5567105e-04,
        1.0942547e-07, 5.6812712e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

* Are there any functions we used? Which are the same? Which haven't you seen before?

**Used functions:**
* `tensor.numpy()`: convert tensor to and ndarray

**Unseen functions:**
* `tf.keras` API